In [3]:
# make the screen bigger!
from IPython.display import display, HTML
display(HTML(data=""" <style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 85%; }
    div#maintoolbar-container { width: 99%; } </style> """))


import numpy as np
import arcpy
import os
from arcpy.sa import *
import pandas as pd
import gdal
from arcpy import env
import arcpy.cartography as CA


arcpy.env.overwriteOutput = True # make sure overwrite files is on

cel_size = 50     # in m 
# projection definition 
sr_project = arcpy.SpatialReference(32702)   # Project dataset into WGS84

In [2]:
GIS_FOLDER = os.path.join('..', 'Raw_GIS_Data')

# path to the grid bound
Grid_shp = os.path.join(GIS_FOLDER, 'grid_bound.shp')

if not os.path.exists(os.path.join('.', 'output//')):
    os.makedirs(os.path.join('.', 'output//'))
                        
if not os.path.exists(os.path.join('.', 'output//Figures//')):
    os.makedirs(os.path.join('.', 'output//Figures//'))
fig_path =  (os.path.join('.', 'output//Figures//'))


# define where the land is too steep to build
steep_land = os.path.join(GIS_FOLDER, 'Slopes', 'steep_land_25deg.shp')

In [ ]:
# project file 
arcpy.Project_management(os.path.join(GIS_FOLDER, "Land_use\\Land_use_wRO_codes2.shp"), os.path.join('.', "WorkSpaces", 'LU_shp_projected.shp'), sr_project)
CA.SimplifyPolygon(os.path.join('.', "WorkSpaces", 'LU_shp_projected.shp'), os.path.join('.', "WorkSpaces", 'LU_shp_projected_simplify.shp'), "POINT_REMOVE", 10, error_option = "RESOLVE_ERRORS")

# Start clipping stuff

In [6]:
Agroforest_buffer = 5
Cultivated_Land_buffer = 4
Developed_Woodlands_buffer = 3
Open_Space_buffer = 1
Impervious_Surfaces_buffer = 0.2 
Buildings_buffer = .5 

In [7]:
def clip_da_use(maptoclip, orig_map_to_buffer, LU_type, buffer_dist_m, steep_land, iteration_num):
    
    tempselect = os.path.join('.', "WorkSpaces\\temp.shp") 
    tempbuffer = os.path.join('.', "WorkSpaces\\temp_buffer.shp")
    buffer_clipped_slopes = os.path.join('.', "WorkSpaces\\temp_buffer_clipped.shp")
    clipped_Map = os.path.join('.', "WorkSpaces\\LU_shp_erased.shp")
    output = os.path.join('.', "WorkSpaces\\intermediate_LU_new_{}.shp".format(iteration_num))
    
    # Buffer the features of interest
    arcpy.Select_analysis(orig_map_to_buffer, tempselect, "LBLCLASS = '{}'".format(LU_type))     # select just the LU type desired
    arcpy.Buffer_analysis(tempselect, tempbuffer, buffer_dist_m, dissolve_option="NONE")
    arcpy.RepairGeometry_management(tempbuffer)
    
    # erase out areas where there are large slopes
    arcpy.Erase_analysis(tempbuffer, steep_land,  buffer_clipped_slopes)
    
    # erase out the new cultivated land zones
    arcpy.Erase_analysis(maptoclip, buffer_clipped_slopes, clipped_Map)    
    
    # add the new land use type back to the big LU map 
    arcpy.Merge_management([buffer_clipped_slopes, clipped_Map], output)

In [9]:
# Agroforest
iteration_num      = 1
maptoclip          = os.path.join('.', "WorkSpaces", 'LU_shp_projected.shp')   # this is where you put in the map from the lst iteration
orig_map_to_buffer = os.path.join('.', "WorkSpaces", 'LU_shp_projected.shp')
LU_type            = 'Agroforest'
buffer_dist_m      = Agroforest_buffer
steep_land         = steep_land

clip_da_use(maptoclip, orig_map_to_buffer, LU_type, buffer_dist_m, steep_land, iteration_num)

In [ ]:
# Cultivated Land
iteration_num      = 2
maptoclip          = os.path.join('.', "WorkSpaces\\intermediate_LU_new_{}.shp".format(iteration_num-1))
orig_map_to_buffer = os.path.join('.', "WorkSpaces", 'LU_shp_projected.shp')
LU_type            = 'Cultivated Land'
buffer_dist_m      = Cultivated_Land_buffer
steep_land         = steep_land

clip_da_use(maptoclip, orig_map_to_buffer, LU_type, buffer_dist_m, steep_land, iteration_num)

In [ ]:
# Developed Woodlands
iteration_num      = 3
maptoclip          = os.path.join('.', "WorkSpaces\\intermediate_LU_new_{}.shp".format(iteration_num-1))
orig_map_to_buffer = os.path.join('.', "WorkSpaces", 'LU_shp_projected.shp')
LU_type            = 'Developed Woodlands'
buffer_dist_m      = Developed_Woodlands_buffer
steep_land         = steep_land

clip_da_use(maptoclip, orig_map_to_buffer, LU_type, buffer_dist_m, steep_land, iteration_num)

In [ ]:
# Open Space
iteration_num      = 4
maptoclip          = os.path.join('.', "WorkSpaces\\intermediate_LU_new_{}.shp".format(iteration_num-1))
orig_map_to_buffer = os.path.join('.', "WorkSpaces", 'LU_shp_projected.shp')
LU_type            = 'Open Space'
buffer_dist_m      = Open_Space_buffer
steep_land         = steep_land

clip_da_use(maptoclip, orig_map_to_buffer, LU_type, buffer_dist_m, steep_land, iteration_num)

In [ ]:
# Impervious Surfaces
iteration_num      = 5
maptoclip          = os.path.join('.', "WorkSpaces\\intermediate_LU_new_{}.shp".format(iteration_num-1))
orig_map_to_buffer = os.path.join('.', "WorkSpaces", 'LU_shp_projected.shp')
LU_type            = 'Impervious Surfaces'
buffer_dist_m      = Impervious_Surfaces_buffer
steep_land         = steep_land

clip_da_use(maptoclip, orig_map_to_buffer, LU_type, buffer_dist_m, steep_land, iteration_num)

In [ ]:
# Buildings
iteration_num      = 6
maptoclip          = os.path.join('.', "WorkSpaces\\intermediate_LU_new_{}.shp".format(iteration_num-1))
orig_map_to_buffer = os.path.join('.', "WorkSpaces", 'LU_shp_projected.shp')
LU_type            = 'Buildings'
buffer_dist_m      = Buildings_buffer
steep_land         = steep_land

clip_da_use(maptoclip, orig_map_to_buffer, LU_type, buffer_dist_m, steep_land, iteration_num)

In [20]:

#  Merge in the grid bound into the shapefile to create accurate grid coverage 
arcpy.Erase_analysis(Grid_shp ,maptoclip,  os.path.join('.', "WorkSpaces", 'LU_shp_bound.shp'))
arcpy.Merge_management([os.path.join('.', "WorkSpaces", 'LU_shp_bound.shp'), maptoclip], os.path.join('.', "WorkSpaces", 'LU_shp_ready.shp'))

# convert to raster.asc
arcpy.PolygonToRaster_conversion(os.path.join('.', "WorkSpaces", 'LU_shp_ready.shp'), "LU2", os.path.join('.', "WorkSpaces", "LU_raster"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size)
arcpy.RasterToASCII_conversion(os.path.join('.', "WorkSpaces", "LU_raster"), os.path.join('.', "WorkSpaces", "new_LU_grid.asc"))



#  Merge in the grid bound into the shapefile to create accurate grid coverage 
arcpy.Erase_analysis(Grid_shp ,orig_map_to_buffer,  os.path.join('.', "WorkSpaces", 'LU_shp_bound.shp'))
arcpy.Merge_management([os.path.join('.', "WorkSpaces", 'LU_shp_bound.shp'), orig_map_to_buffer], os.path.join('.', "WorkSpaces", 'LU_shp_ready.shp'))

# convert to raster.asc
arcpy.PolygonToRaster_conversion(os.path.join('.', "WorkSpaces", 'LU_shp_ready.shp'), "LU2", os.path.join('.', "WorkSpaces", "LU_raster"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size)
arcpy.RasterToASCII_conversion(os.path.join('.', "WorkSpaces", "LU_raster"), os.path.join('.', "WorkSpaces", "original_LU_grid.asc"))

<Result '.\\input\\General\\original_LU_grid.asc'>

'.\\input/General'

In [16]:
arr = arcpy.RasterToNumPyArray(os.path.join('.', "WorkSpaces", "original_LU_grid.asc"),  nodata_to_value=-999)
unique, counts = np.unique(arr, return_counts=True)
arr_stats_original = np.asarray((unique, counts)).T

arr = arcpy.RasterToNumPyArray(os.path.join('.', "WorkSpaces", "new_LU_grid.asc"),  nodata_to_value=-999)
unique, counts = np.unique(arr, return_counts=True)
arr_stats_new = np.asarray((unique, counts)).T

for idx, val in enumerate(arr_stats_original):
    oabs = arr_stats_original[idx][1]
    nabs = arr_stats_new[idx][1]
    
    pctchange = ((nabs/oabs)-1)*100
    print("LU type_{:.0f} is {:.2f}% different".format(val[0], pctchange))
   
    

LU type_0 is -0.03% different
LU type_400 is 15.12% different
LU type_500 is -44.77% different
LU type_700 is 207.08% different
LU type_800 is -56.14% different
LU type_1000 is -49.55% different
LU type_1200 is -18.37% different
LU type_1300 is -20.07% different
LU type_1400 is -40.03% different
LU type_1500 is -2.44% different
LU type_1600 is -2.48% different
LU type_1700 is -32.01% different
LU type_2100 is 0.00% different


In [19]:
LK = pd.read_csv(os.path.join(GIS_FOLDER, 'Land_use', 'Lookup_num_to_name.csv'))
LK


,Tutuila Description,Land Use Code
0,Cultivated Land,400
1,Agroforest,500
2,Open Space,700
3,Developed Woodlands,800
4,"Buildings, Impervious Surfaces",1000
5,"Marsh, Mangroves, Swamp",1200
6,"Landslips/Clearcuts, Landfill, Quarries and Gr...",1300
7,Grassland/Herbaceous,1400
8,"Upland Scrub, Coastal Scrub",1500
9,"Lowland Rainforest, Coastal Forest",1600


In [22]:
LU type_0 is -0.03% different
LU type_400 is 15.12% different
LU type_500 is -44.77% different
LU type_700 is 207.08% different
LU type_800 is -56.14% different
LU type_1000 is -49.55% different
LU type_1200 is -18.37% different
LU type_1300 is -20.07% different
LU type_1400 is -40.03% different
LU type_1500 is -2.44% different
LU type_1600 is -2.48% different
LU type_1700 is -32.01% different
LU type_2100 is 0.00% different

LU type_0 is -0.03% different
LU type_400 is 15.12% different
LU type_500 is -44.77% different
LU type_700 is 207.08% different
LU type_800 is -56.14% different
LU type_1000 is -49.55% different
LU type_1200 is -18.37% different
LU type_1300 is -20.07% different
LU type_1400 is -40.03% different
LU type_1500 is -2.44% different
LU type_1600 is -2.48% different
LU type_1700 is -32.01% different
LU type_2100 is 0.00% different
